Loading Libraries and Reading Data In

In [1]:
# import packages for analysis and modeling
import pandas as pd  # data frame operations
import pandas as pd #df operations
import numpy as np #arrays and math functions
import statsmodels.api as sm #stat models (regression)
import matplotlib.pyplot as plt #2d plotting
from pandas.tools.plotting import scatter_matrix  # scatter plot matrix
import numpy as np  # arrays and math functions
from scipy.stats import uniform  # for training-and-test split
import statsmodels.api as sm  # statistical models (including regression)
import statsmodels.formula.api as smf  # R-like model specification
from sklearn.tree import DecisionTreeRegressor  # machine learning tree
from sklearn.ensemble import RandomForestRegressor # ensemble method
from sklearn.preprocessing import Imputer
%config IPCompleter.greedy=True #tabbing for autocomplete
import seaborn as sns  # PROVIDES TRELLIS AND SMALL MULTIPLE PLOTTING


#read in data from csv and assign index
coaches = pd.read_csv('/Users/jdine/Desktop/SYracuse/Term 5 - Current/Advanced Info Analytics- IST 718/Homework/Coaches.csv.')
coaches['idx'] = range(len(coaches))  # for use as index
coaches = coaches.set_index(['idx']) 

coaches.df = pd.DataFrame(coaches)

ncaa = pd.read_csv('/Users/jdine/Desktop/SYracuse/Term 5 - Current/Advanced Info Analytics- IST 718/Homework/NCAA.csv.')
ncaa.df = pd.DataFrame(ncaa)





C:\Users\jdine\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Cleaning Data - Coaches

In [2]:
# check the structure of the coaches data frame 
print(coaches.df.shape)  #124 obeservations by 7 feature


nullcounts = coaches.df.isnull().sum() #Number of missing values in the coaches dataframe per column
print(nullcounts) #8 coaches have no salary listed


#Replace OtherPay and Maxbonus NA/s with average of column
coaches.df["OtherPay"].fillna(coaches.df["OtherPay"].mean(),inplace=True)
coaches.df["MaxBonus"].fillna(coaches.df["MaxBonus"].mean(), inplace=True)


#Print Averages of all Salary Columns
print(np.mean(coaches.df.Salary)) #Average Salary 
print(np.mean(coaches.df.OtherPay)) #Average Other Pay
print(np.mean(coaches.df.TotalSalary)) #Average Total Salary


#Recheck NA Count after cleansing
coaches.df = coaches.df[np.isfinite(coaches.df['TotalSalary'])] #remove 8 obs. where salary is not listed (won't help with regression)
print(coaches.df.shape)

nullcounts = coaches.df.isnull().sum() #Number of missing values in the coaches dataframe per column
print(nullcounts) #8 coaches have no salary listed






(124, 7)
Name            0
School          0
Conference      0
Salary          8
OtherPay       50
TotalSalary     8
MaxBonus       20
dtype: int64
1581876.7931034483
25837.202702702685
1598359.146551724
(116, 7)
Name           0
School         0
Conference     0
Salary         0
OtherPay       0
TotalSalary    0
MaxBonus       0
dtype: int64


Cleaning Data - NCAA

In [3]:
# check the structure of the coaches data frame 
print(ncaa.df.shape)  #345 obeservations by 227 feature

ncaa.df = ncaa.df.filter(regex='2006|SCL') #keeping data from year = 2006 (.filter + regex (contains))
ncaa.df = pd.DataFrame(ncaa.df) #store result as dataframe



ncaa.df["School"] = ncaa.df["SCL_NAME"]
ncaa.df.drop(['SCL_UNITID'], axis = 1, inplace = True) #SCL UnitID is useless index column. Drop
ncaa.df.drop(['SCL_NAME'], axis = 1, inplace = True) #SCL UnitID is useless index column. Drop


nullcounts.ncaa = ncaa.df.isnull().sum() #Number of missing values 
print(nullcounts.ncaa) #Checking Nulls

ncaa.df #Call df



(345, 226)
SCL_DIVISION               0
SCL_SUBDIVISION            0
SCL_CONFERENCE             0
SCL_HBCU                   0
SCL_PRIVATE                0
FED_N_2006_SA              0
FED_RATE_2006_SA           0
GSR_N_2006_SA              0
GSR_2006_SA                0
FED_RATE_MALE_2006_SA      0
FED_RATE_FEMALE_2006_SA    0
GSR_MALE_2006_SA           0
GSR_FEMALE_2006_SA         0
FED_RATE_2006_SB           0
FED_RATE_MALE_2006_SB      0
FED_RATE_FEMALE_2006_SB    0
School                     0
dtype: int64


,SCL_DIVISION,SCL_SUBDIVISION,SCL_CONFERENCE,SCL_HBCU,SCL_PRIVATE,FED_N_2006_SA,FED_RATE_2006_SA,GSR_N_2006_SA,GSR_2006_SA,FED_RATE_MALE_2006_SA,FED_RATE_FEMALE_2006_SA,GSR_MALE_2006_SA,GSR_FEMALE_2006_SA,FED_RATE_2006_SB,FED_RATE_MALE_2006_SB,FED_RATE_FEMALE_2006_SB,School
0,1,2,Southwestern Athletic Conf.,1,0,54,43,53,47,41,47,47,47,22,20,23,Alabama A & M University
1,1,1,Conference USA,2,0,54,57,77,66,48,68,59,81,48,47,49,University of Alabama at Birmingham
2,1,2,Southwestern Athletic Conf.,1,0,59,37,36,89,41,33,90,88,26,19,32,Alabama State University
3,1,1,Southeastern Conference,2,0,88,70,75,92,56,84,84,100,67,64,69,The University of Alabama
4,1,1,Southeastern Conference,2,0,101,65,93,80,56,77,67,95,68,66,70,Auburn University
5,1,2,Ohio Valley Conference,2,0,69,45,69,62,45,45,58,68,29,22,34,Jacksonville State University
6,1,2,Southern Conference,2,1,58,79,55,100,76,83,100,100,71,66,75,Samford University
7,1,1,Sun Belt Conference,2,0,31,68,47,74,50,76,60,85,37,33,39,University of South Alabama
8,1,1,Sun Belt Conference,2,0,58,55,83,71,45,67,65,83,35,28,40,Troy University
9,1,1,Pac-12 Conference,2,0,79,56,69,80,49,63,76,84,57,54,59,Arizona State University-Tempe


Merging Data -- GradRates

In [22]:
#Matching: https://github.com/RobinL/fuzzymatcher/blob/master/examples.ipynb
!pip install python_Levenshtein-0.12.0-cp36-cp36m-win_amd64.whl 
##need to downloadLevenshtein-0.12.0-cp36-cp36m-win_amd64.whl from https://www.lfd.uci.edu/~gohlke/pythonlibs/#python-levenshtein
!pip install fuzzymatcher
import fuzzymatcher

left_on = ["School", "Conference"] 
right_on = ["School", "SCL_CONFERENCE"]


dfjoined = fuzzymatcher.fuzzy_left_join(coaches.df,ncaa.df, left_on, right_on) #Merge on School and Conference
dfjoined = pd.DataFrame(dfjoined) #Dataframe creation
dfjoined = dfjoined.sort_values(by='best_match_score', ascending = False) #
dfjoined = dfjoined.query('best_match_score > -1') #removing rows where best_match_score < 0

col= dfjoined.columns.tolist()
col

#list of current columns
cols =['best_match_score',
 '__id_left',
 '__id_right',
 'Name',
 'School_right',
 'School_left',
 'Conference',
 'Salary',
 'OtherPay',
 'TotalSalary',
 'MaxBonus',
 'SCL_DIVISION',
 'SCL_SUBDIVISION',
 'SCL_CONFERENCE',
 'SCL_HBCU',
 'SCL_PRIVATE',
 'FED_N_2006_SA',
 'FED_RATE_2006_SA',
 'GSR_N_2006_SA',
 'GSR_2006_SA',
 'FED_RATE_MALE_2006_SA',
 'FED_RATE_FEMALE_2006_SA',
 'GSR_MALE_2006_SA',
 'GSR_FEMALE_2006_SA',
 'FED_RATE_2006_SB',
 'FED_RATE_MALE_2006_SB',
 'FED_RATE_FEMALE_2006_SB']


dfjoined = dfjoined['Name',
 'School_left',
 'Conference',
 'TotalSalary',
 'SCL_DIVISION',
 'SCL_SUBDIVISION',
 'SCL_CONFERENCE',
 'SCL_HBCU',
 'SCL_PRIVATE',
 'FED_N_2006_SA',
 'FED_RATE_2006_SA',
 'GSR_N_2006_SA',
 'GSR_2006_SA',
 'FED_RATE_2006_SB',]

dfjoined




#df with grad rates and coaches data = dfjoined.


#load in data pertaining to stadium size
stadiums = pd.read_csv('/Users/jdine/Desktop/SYracuse/Term 5 - Current/Advanced Info Analytics- IST 718/Homework/NCAAstadiums.csv.')
stadiums.head()
dfjoined.head()

l_on = ["School_left", "Conference"] 
r_on = ["School", "Conference"]

dfjoined = fuzzymatcher.fuzzy_left_join(dfjoined,stadiums, l_on, r_on) #Merge on School and Conference





KeyError: ('Name', 'School_left', 'Conference', 'TotalSalary', 'SCL_DIVISION', 'SCL_SUBDIVISION', 'SCL_CONFERENCE', 'SCL_HBCU', 'SCL_PRIVATE', 'FED_N_2006_SA', 'FED_RATE_2006_SA', 'GSR_N_2006_SA', 'GSR_2006_SA', 'FED_RATE_2006_SB')

Merging Data on stadium

In [12]:
l_on = ["School_left", "Conference"] 
r_on = ["School", "Conference"]


dfjoined



,best_match_score,__id_left,__id_right,Name,School_left,Conference,Salary,OtherPay,TotalSalary,MaxBonus,...,GSR_N_2006_SA,GSR_2006_SA,FED_RATE_MALE_2006_SA,FED_RATE_FEMALE_2006_SA,GSR_MALE_2006_SA,GSR_FEMALE_2006_SA,FED_RATE_2006_SB,FED_RATE_MALE_2006_SB,FED_RATE_FEMALE_2006_SB,School_right
156,0.633704,47_left,336_right,Dana Holgorsen,West Virginia,Big 12,2380000.0,0.000000,2380000.0,6.000000e+05,...,71.0,92.0,61.0,71.0,90.0,94.0,56.0,53.0,60.0,West Virginia University
14,0.554212,6_left,122_right,Todd Berry,Louisiana-Monroe,Sun Belt,250000.0,25837.202703,250000.0,6.250000e+04,...,60.0,47.0,27.0,55.0,37.0,71.0,37.0,33.0,38.0,University of Louisiana at Monroe
152,0.552637,44_left,167_right,Bobby Hauck,Nevada-Las Vegas,Mt. West,500000.0,900.000000,500900.0,1.400000e+05,...,62.0,74.0,62.0,64.0,67.0,83.0,41.0,38.0,44.0,University of Nevada-Las Vegas
314,0.552459,55_left,7_right,Joey Jones,South Alabama,Sun Belt,345125.0,26000.000000,371125.0,5.062500e+04,...,47.0,74.0,50.0,76.0,60.0,85.0,37.0,33.0,39.0,University of South Alabama
244,0.534143,51_left,126_right,Mark Hudspeth,Louisiana-Lafayette,Sun Belt,800000.0,3000.000000,803000.0,1.050000e+05,...,74.0,80.0,70.0,83.0,79.0,81.0,44.0,39.0,48.0,University of Louisiana at Lafayette
699,0.499488,111_left,308_right,Tommy Tuberville,Texas Tech,Big 12,2150000.0,5000.000000,2155000.0,1.025000e+06,...,77.0,75.0,52.0,73.0,68.0,89.0,62.0,60.0,64.0,Texas Tech University
50,0.490059,22_left,148_right,Mark Dantonio,Michigan State,Big Ten,1934250.0,0.000000,1934250.0,6.500000e+05,...,83.0,87.0,72.0,74.0,80.0,95.0,79.0,78.0,81.0,Michigan State University
321,0.454269,61_left,333_right,Mike Leach,Washington State,PAC-12,2250000.0,0.000000,2250000.0,6.400000e+05,...,73.0,79.0,66.0,76.0,74.0,84.0,67.0,64.0,71.0,Washington State University
507,0.438151,84_left,305_right,Gary Patterson,Texas Christian,Big 12,3467926.0,25837.202703,3467926.0,5.230432e+05,...,65.0,94.0,79.0,73.0,93.0,96.0,75.0,74.0,76.0,Texas Christian University
91,0.435594,36_left,90_right,Pat Fitzgerald,Northwestern,Big Ten,1280751.0,25837.202703,1280751.0,5.230432e+05,...,93.0,97.0,91.0,91.0,95.0,98.0,93.0,92.0,94.0,Northwestern University


Summary Statistics and Visuals

In [ ]:
len(dfjoined) #87 rows of merged data remaining. Started with 124 coaches

#print summary statistics
print(dfjoined.describe())

#simple boxplot for TotalSalary
sns.boxplot(y= "TotalSalary", data= dfjoined)
plt.show()

#Create correlation matrix - Code borrowed from week2 assignments
corr = dfjoined.corr()
# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))
# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.show()

#Lattice plots w/ various yaxes - Code borrowed from week2 assignments
sns.set(style="darkgrid")
g = sns.FacetGrid(dfjoined, col="SCL_PRIVATE",                   
                  hue_kws=dict(marker=["^", "v"]))
g.map(plt.scatter, "TotalSalary", "FED_N_2006_SA", alpha=.7)
g.add_legend();
plt.show()

sns.set(style="darkgrid")
g = sns.FacetGrid(dfjoined, col="SCL_PRIVATE",                   
                  hue_kws=dict(marker=["^", "v"]))
g.map(plt.scatter, "TotalSalary", "FED_RATE_2006_SA", alpha=.7)
g.add_legend();
plt.show()

sns.set(style="darkgrid")
g = sns.FacetGrid(dfjoined, col="SCL_PRIVATE",                   
                  hue_kws=dict(marker=["^", "v"]))
g.map(plt.scatter, "TotalSalary", "GSR_2006_SA", alpha=.7)
g.add_legend();
plt.show()







In [ ]:
#build a model to show relevant variables
my_model = str('TotalSalary ~ Conference + FED_N_2006_SA + FED_RATE_2006_SA +FED_RATE_2006_SA + GSR_N_2006_SA+GSR_2006_SA + FED_RATE_2006_SB')

train_model_fit = smf.ols(my_model, data = dfjoined).fit()
print(train_model_fit.summary())
